In [4]:
# app_link_prod.py — ultra-minimal Plaid prod server

import os, json, secrets
from pathlib import Path
from flask import Flask, jsonify, request
from dotenv import load_dotenv

# ---- Plaid SDK ----
from plaid import ApiClient, Configuration, ApiException
from plaid.api import plaid_api
from plaid import Environment as PlaidEnv
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.products import Products
from plaid.model.country_code import CountryCode
from plaid.model.item_public_token_exchange_request import ItemPublicTokenExchangeRequest

# --- Load .env with override ---
REPO = Path().resolve()
load_dotenv(REPO / ".env", override=True)

# --- Required env (production by default) ---
PLAID_ENV = (os.getenv("PLAID_ENV") or "production").strip().lower()
if PLAID_ENV not in ("production", "sandbox"):
    raise RuntimeError("PLAID_ENV must be 'production' or 'sandbox'")

CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
SECRET    = os.getenv("PLAID_SECRET")
if not CLIENT_ID or not SECRET:
    raise RuntimeError("Missing PLAID_CLIENT_ID or PLAID_SECRET")

# Optional (needed for OAuth institutions)
PLAID_REDIRECT_URI = os.getenv("PLAID_REDIRECT_URI") or ""

# --- Map env to Plaid host (only sandbox/production) ---
PLAID_HOST = PlaidEnv.Production if PLAID_ENV == "production" else PlaidEnv.Sandbox
PLAID_BASE_URL = "https://production.plaid.com" if PLAID_ENV == "production" else "https://sandbox.plaid.com"

# --- Plaid client ---
configuration = Configuration(host=PLAID_HOST, api_key={"clientId": CLIENT_ID, "secret": SECRET})
client = plaid_api.PlaidApi(ApiClient(configuration))

# --- Flask app ---
app = Flask(__name__)

@app.get("/debug/env")
def debug_env():
    # This is the single source of truth: must say 'production'
    return jsonify({
        "PLAID_ENV": PLAID_ENV,
        "PLAID_BASE_URL": PLAID_BASE_URL,
        "CLIENT_ID_set": bool(CLIENT_ID),
        "SECRET_set": bool(SECRET),
        "PLAID_REDIRECT_URI": PLAID_REDIRECT_URI or None
    })

@app.post("/create_link_token")
def create_link_token():
    # Always mint a new token after any env change
    req = LinkTokenCreateRequest(
        client_name="Blue Lantern Dashboard",
        user=LinkTokenCreateRequestUser(client_user_id=secrets.token_hex(8)),
        products=[Products("transactions")],
        country_codes=[CountryCode("US")],
        language="en",
        redirect_uri=PLAID_REDIRECT_URI or None  # safe in both envs
    )
    try:
        resp = client.link_token_create(req)
        return jsonify({"link_token": resp.to_dict().get("link_token")})
    except ApiException as e:
        try:
            return jsonify({"error": json.loads(e.body)}), 400
        except Exception:
            return jsonify({"error": str(e)}), 400

@app.post("/exchange_public_token")
def exchange_public_token():
    data = request.get_json(silent=True) or {}
    public_token = data.get("public_token")
    if not public_token:
        return jsonify({"error": "missing public_token"}), 400
    try:
        resp = client.item_public_token_exchange(
            ItemPublicTokenExchangeRequest(public_token=public_token)
        )
        d = resp.to_dict()
        # Return only the essentials; you’ll store access_token in your app
        return jsonify({"ok": True, "item_id": d.get("item_id")})
    except ApiException as e:
        try:
            return jsonify({"error": json.loads(e.body)}), 400
        except Exception:
            return jsonify({"error": str(e)}), 400

# OAuth return endpoint (no template—just something to land on)
@app.get("/oauth-return")
def oauth_return():
    return jsonify({"ok": True, "message": "Back from institution OAuth", "env": PLAID_ENV})

if __name__ == "__main__":
    # Bind to localhost; expose via ngrok https
    app.run(host="127.0.0.1", port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


2025-09-07 13:19:20,474 | INFO | WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-09-07 13:19:20,475 | INFO | Press CTRL+C to quit
